### This notebook demonstrates how to evaluate accuracy

In [ ]:
from sklearn.neighbors import NearestNeighbors
from nltk.stem.porter import PorterStemmer
from BM25Vectorizer import BM25Vectorizer
from nltk.corpus import stopwords 
from tqdm import tqdm
from utils import *

data, data_train, data_seen_500, data_unseen_500, data_test = load_data()

### Test description set (200) data

In [ ]:
vectorizers = [BM25Vectorizer(stop_words=stopwords.words('english'), stemmer=PorterStemmer(), norm=False),
               BM25Vectorizer(stop_words=stopwords.words('english'), stemmer=None, norm=False),
               BM25Vectorizer(stop_words=None, stemmer=PorterStemmer(), norm=False),
               BM25Vectorizer(stop_words=None, stemmer=None, norm=False)]

for vectorizer in vectorizers:
    print('=================================')
    print(f'stop_words: {vectorizer.stop_words != None}, stemmer: {vectorizer.stemmer != None}')

    X_BM25 = vectorizer.fit_transform(data_train) # <--- fit train data here
    words = vectorizer.words

    knn = NearestNeighbors(metric='cosine')
    knn.fit(X_BM25)

    y_pred = []
    y_gold = []
    for word, defi in tqdm(data_unseen_500.items()): # <--- use unseen_500 data here
        query = list(defi)[0]
        query = vectorizer.transform(query)

        prediction = search(query, knn, words, n=1000)

        y_pred.append(prediction)
        y_gold.append(word)

    evaluate(y_pred, y_gold)